In [2]:
import os

current_working_directory = os.getcwd()

print("Current working directory:", current_working_directory)

Current working directory: /home/kenchen1216/StockTools/US_Stock


In [ ]:
Test Pytorch

In [1]:
import torch
print(torch.cuda.is_available()) # Should be True
print(torch.cuda.device_count()) # Should be > 0

True
1


In [2]:
!nvidia-smi

Fri Mar 29 19:28:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 3000                On  |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              4W /   80W |      13MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
使用AutoGluon訓練模型
波段交易的目標是捕捉股票價格的中短期波動

In [2]:
import pandas as pd
import os
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

# 步骤1: 合并CSV文件
def combine_csv_files(folder_path):
    combined_df = pd.DataFrame()
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            temp_df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, temp_df], ignore_index=True)
    combined_df = combined_df.dropna()
    return combined_df

# 步骤3: 分割数据集
def split_data(df):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    return train_df, test_df

# 步骤4: 使用AutoGluon训练模型并启用GPU加速
def train_model(train_df, label_column='Future_Signal'):
    model_path = '/home/kenchen1216/StockTools/US_Stock/models/2d_Future_model3'
    predictor = TabularPredictor(label=label_column, path=model_path).fit(train_data=train_df, presets='best_quality', num_gpus=1 ,
                                                                          hyperparameters='extreme') # Grant 1 gpu for the entire Tabular Predictor
    return predictor

# 步骤5: 评估模型
def evaluate_model(predictor, test_df):
    predictor.leaderboard(test_df)
    performance = predictor.evaluate(test_df)
    print(performance)

# 主程序
if __name__ == "__main__":
    folder_path = '/home/kenchen1216/StockTools/US_Stock/ProcessData/ProcessData 3'  # 修改为你的文件夹路径
    combined_df = combine_csv_files(folder_path)
    train_df, test_df = split_data(combined_df)
    predictor = train_model(train_df, label_column='Future_Signal')
    evaluate_model(predictor, test_df)


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: /home/kenchen1216/StockTools/US_Stock/models/2d_Future_model3/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "/home/kenchen1216/StockTools/US_Stock/models/2d_Future_model3/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9

We have to download the TabPFN, as there is no checkpoint at  /home/kenchen1216/miniconda3/envs/AutoGluon/lib/python3.9/site-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt
It has about 100MB, so this might take a moment.


		[enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 127838715904 bytes. Error code 12 (Cannot allocate memory)
Detailed Traceback:
Traceback (most recent call last):
  File "/home/kenchen1216/miniconda3/envs/AutoGluon/lib/python3.9/site-packages/autogluon/core/trainer/abstract_trainer.py", line 1817, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "/home/kenchen1216/miniconda3/envs/AutoGluon/lib/python3.9/site-packages/autogluon/core/trainer/abstract_trainer.py", line 1763, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, total_resources=total_resources, **model_fit_kwargs)
  File "/home/kenchen1216/miniconda3/envs/AutoGluon/lib/python3.9/site-packages/autogluon/core/models/abstract/abstract_model.py", line 854, in fit
    out = self._fit(**kwargs)
  File "/home/kenchen1216/miniconda3/envs/AutoGluon/lib/python3.9

{'accuracy': 0.6586860946659279, 'balanced_accuracy': 0.28059334096779304, 'mcc': 0.16846901763989977}


In [ ]:
使用AutoGluon訓練模型
時間序列預測

In [7]:
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import pandas as pd
import os

# 設定目錄路徑
directory_path = '/home/kenchen1216/StockTools/US_Stock/Daily_K'
output_directory_path = '/home/kenchen1216/StockTools/US_Stock/models/TimeSeries'  # 預測結果儲存的目錄
predict_path = '/home/kenchen1216/StockTools/US_Stock/TWPredictedData/TimeSeries'

# 確保輸出目錄存在
os.makedirs(output_directory_path, exist_ok=True)

# 遍歷目錄中的所有CSV檔案
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        print(f"處理檔案: {file_path}")

        # 載入數據
        data = pd.read_csv(file_path)

        # 預處理（確保時間戳的格式正確）
        data['timestamp'] = pd.to_datetime(data['Date']).dt.tz_localize(None)
        data.drop(columns=['Date'], inplace=True)

        data = TimeSeriesDataFrame.from_data_frame(data, id_column="item_id", timestamp_column="timestamp") 
        model_path = os.path.join(output_directory_path, filename)
        os.makedirs(model_path, exist_ok=True)
        # 初始化時間序列預測器，設置預測未來3天
        predictor = TimeSeriesPredictor(
            path=model_path,  # 為每個檔案創建獨立的模型目錄
            prediction_length=3,  # 預測未來3天的收盤價
            target='Close',  # 我们要预测的目标变量
            eval_metric="MAPE",
            freq='D'  # 假設為每日數據。根據您數據的頻率調整此設定
        )

        # 訓練模型
        predictor.fit(
            train_data=data,
            hyperparameters='default',  # 使用預設的超參數
            presets="high_quality"
        )

        # 進行預測
        forecast = predictor.predict(data.tail(predictor.prediction_length))

        # 將預測結果保存到CSV檔案中
        output_file_path = os.path.join(predict_path, filename)
        forecast.to_csv(output_file_path, index=False)

        print(f"預測結果已儲存到 {output_file_path}")


Beginning AutoGluon training...
AutoGluon will save models to 'output_directory_path/3432.TW.csv'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jan 11 04:09:03 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       29.85 GB / 31.24 GB (95.6%)
Disk Space Avail:   877.23 GB / 1006.85 GB (87.1%)
Setting presets to: high_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 3,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'Close',
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency 'D'.


處理檔案: /home/kenchen1216/StockTools/US_Stock/Daily_K/3432.TW.csv


train_data contains missing values represented by NaN. They have been filled by carrying forward the last valid observation.
Provided train_data has 4559 rows, 1 time series. Median time series length is 4559 (min=4559, max=4559). 

Provided dataset contains following columns:
	target:           'Close'

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-03-30 19:54:37
Models that will be trained: ['SeasonalNaive', 'CrostonSBA', 'NPTS', 'AutoETS', 'DynamicOptimizedTheta', 'AutoARIMA', 'RecursiveTabular', 'DirectTabular', 'DeepAR', 'TemporalFusionTransformer', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-0.0282       = Validation score (-MAPE)
	0.01    s     = Training runtime
	1.25    s     = Validation (prediction) runtime
Training timeseries model CrostonSBA. 
	-0.0426    

PermissionError: [Errno 13] Permission denied: '/home/kenchen1216/StockTools/US_Stock/TWPredictedData/TimeSeries/3432.TW.csv'